In [1]:
import pandahouse as ph
import pandas as pd
import datetime

In [2]:
# создаем словарь connection с нужными параметрами
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

### 2.1 Очень усердные ученики.
Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".<br>
<br>
Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

In [3]:
# Создаю SQL запрос. 
# В подзапросе вывожу id студентов, месяц и количество успешно выполненных горошин. в HAVING фильтрую количество горошин(от 20)
# В основном запросе просто вывожу количество ID из подзапроса.
q_peas = '''
    SELECT
        count(st_id) as exelent_students
    FROM
        (SELECT 
            st_id,
            month(timest) as month,
            count(correct) as correct_tasks
        FROM 
            default.peas
        WHERE
            correct = 1
        GROUP BY
            st_id,
            month(timest)
        HAVING
            count(correct) >= 20)
    '''

In [4]:
# Отправляю запрос и записываю в датафрейм
exelent_students = ph.read_clickhouse(q_peas, connection=connection_default)
exelent_students

,exelent_students
0,136


### 2.2 Оптимизация воронки
Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.<br>
<br>
Необходимо в <b>одном запросе</b> выгрузить следующую информацию о группах пользователей:<br>
- ARPU<br> 
- ARPAU <br>
- CR в покупку<br> 
- СR активного пользователя в покупку<br> 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике<br>
<br>
<b>Активным</b> считается пользователь, за все время решивший <b>больше 10</b> задач правильно в любых дисциплинах.<br>
<b>Активным по математике</b> считается пользователь, за все время решивший <b>2 или больше</b> задач правильно по математике.

In [5]:
# Создаю SQL запрос. 
# В блоке WITH сохраняю отдельно подзапросы:
#    activ - вывод активных пользователей и групп (фильтрую по решенным задачам, в HAVING фильтр по количеству - больше 10); 
#    activ_math - вывод активных пользователей по математике и групп (фильтрую по решенным задачам и типу задачи Math, 
#                 в HAVING фильтр по количеству - больше или равно 2);
# 
# В основном запросе   
# arpu - отношение дохода к числу пользователей;
# arpau - отношение дохода от активных пользователей к числу активных пользователей;
# cr - отношение количества покупателей к числу пользователей;
# cr_activ - отношение количества покупателей из числа активных пользователей к числу активных пользователей;
# cr_math_activ - отношение количества покупателей из числа активных пользователей по математике 
#                 к числу активных пользователей по математике;
#    
# (из каунта пользователей вычитаю единицу, так как при подсчете уникальных значений так же учитываются пропуски)


q_scool = '''
    WITH 
            activ as 
                (SELECT 
                    distinct peas.st_id
                FROM 
                    default.peas
                    JOIN
                    default.studs
                        ON peas.st_id = studs.st_id
                WHERE 
                    correct = 1
                GROUP BY 
                    peas.st_id
                HAVING 
                    count(correct) > 10),
            activ_math as
                (SELECT 
                    distinct peas.st_id
                FROM 
                    default.peas
                    JOIN
                    default.studs
                        ON peas.st_id = studs.st_id
                WHERE 
                    correct = 1 
                    and
                    subject = 'Math'
                GROUP BY 
                    peas.st_id
                HAVING 
                    count(correct) >= 2)


    SELECT
        s.test_grp as test_grp,
        round((sum(ch.money) / count(distinct s.st_id)), 2) as arpu,
        round((sumIf(ch.money, ch.st_id == a.st_id) / (count(distinct a.st_id) - 1)), 2) as arpau,
        round((((count(distinct ch.st_id) - 1) / count(distinct s.st_id)) * 100), 2) as cr,
        round((((countIf(distinct a.st_id, ch.money > 0) - 1) / (count(distinct a.st_id) - 1))* 100), 2) as cr_activ,
        round(((countIF(distinct am.st_id, ch.money > 0 AND ch.subject = 'Math') - 1) / (count(distinct am.st_id) - 1) * 100), 2) as cr_math_activ
    FROM
        default.studs as s
        FULL JOIN
        default.final_project_check as ch
            ON s.st_id = ch.st_id
        FULL JOIN
        activ as a
            ON a.st_id = s.st_id
        FULL JOIN
        activ_math as am
            ON am.st_id = s.st_id
    GROUP BY
        s.test_grp
    '''

In [6]:
scool = ph.read_clickhouse(q_scool, connection=connection_default)
scool

,test_grp,arpu,arpau,cr,cr_activ,cr_math_activ
0,control,4540.98,10393.70,4.92,11.02,6.12
1,pilot,11508.47,29739.58,10.85,26.04,9.52
